In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time, json
import sys
sys.path.append("../../")
import os
import torch
import numpy as np
from tqdm import tqdm

import logging
from src.utils import logging_utils
from src.utils import env_utils, experiment_utils
from src import functional
import wandb

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-29 20:14:35 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
from src.dataset_manager import DatasetManager
DatasetManager.list_datasets_by_group()

2024-10-29 20:14:35 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-10-29 20:14:35 numexpr.utils INFO     NumExpr defaulting to 8 threads.
2024-10-29 20:14:35 datasets INFO     PyTorch version 2.5.0 available.


{'geometry_of_truth': ['sp_en_trans',
  'cities',
  'neg_cities',
  'smaller_than',
  'larger_than',
  'common_claim_true_false',
  'companies_true_false'],
 'relations': ['commonsense/word_sentiment',
  'commonsense/fruit_outside_color',
  'commonsense/task_done_by_person',
  'commonsense/work_location',
  'commonsense/task_done_by_tool',
  'commonsense/substance_phase',
  'commonsense/object_superclass',
  'factual/country_capital_city',
  'factual/person_plays_pro_sport',
  'factual/country_language',
  'factual/country_largest_city',
  'factual/food_from_country',
  'factual/landmark_in_country',
  'factual/superhero_archnemesis',
  'factual/city_in_country',
  'factual/superhero_person',
  'factual/person_plays_instrument',
  'factual/country_currency',
  'factual/person_plays_position_in_sport',
  'linguistic/word_last_letter',
  'linguistic/verb_past_tense',
  'linguistic/word_first_letter',
  'linguistic/adj_superlative',
  'linguistic/adj_comparative',
  'linguistic/adj_antony

In [35]:
dataloader = DatasetManager.from_named_datasets(
    # [("geometry_of_truth", "cities")],
    # [("sst2", "sst2")],
    # [("relations", 'factual/country_capital_city')],
    # [("tense", "tense")],
    [("language_identification", "language_identification")],
    batch_size=5
)
batch = next(iter(dataloader))
batch

2024-10-29 20:34:42 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/FrancophonIA/WiLI-2018 HTTP/11" 200 1480
2024-10-29 20:34:42 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2024-10-29 20:34:42 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/FrancophonIA/WiLI-2018/FrancophonIA/WiLI-2018.py HTTP/11" 404 0
2024-10-29 20:34:42 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/FrancophonIA/WiLI-2018 HTTP/11" 200 1480
2024-10-29 20:34:42 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-10-29 20:34:42 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/FrancophonIA/WiLI-2018/resolve/d505038b20006eb17994578c1e870d443f1cd5d2/README.md HTTP/11" 200 0
2024-10-29 20:34:42 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-10-29 20:34

[ContextQASample(context='o hospital de caridade hc cujo nome oficial é imperial hospital de caridade ihc é um hospital filantrópico de florianópolis localizado no centro é o hospital mais antigo da cidade e do estado de santa catarina além de ficar numa localização simbólica e ser parte de tradições religiosas da cidade', questions=['# Is the text presented in Portugese?', '# Would you identify this text as being in Portugese?', '# Is the language of this text Portugese?', '# Can we determine that this is in Dutch?', '# Am I correct in saying this is in Portugese?', '# Can we say this is in Persian?', '# Does this passage use Urdu?', '# Is the language of this text Estonian?', '# Does this passage use Portugese?', '# Is the content of this text in Latin?'], answers=['Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No']),
 ContextQASample(context='la terminarea celuli de-al doilea război mondial a existat o înțelegere între aliați cu privire la ocuparea temporară a teritorii

In [33]:
from src.dataset_manager import DatasetLoader, ContextQASample, NUM_QA_PER_SAMPLE, YES_TOKEN, NO_TOKEN
import os
from src.utils import env_utils
import json
import random
from datasets import load_dataset

# from src.dataset_manager import TenseDatasetLoader
from src.dataset_manager import LanguageIDDatasetLoader

loader = LanguageIDDatasetLoader()
ds = loader.load()

2024-10-29 20:34:09 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/FrancophonIA/WiLI-2018 HTTP/11" 200 1480
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443


2024-10-29 20:34:09 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/FrancophonIA/WiLI-2018/FrancophonIA/WiLI-2018.py HTTP/11" 404 0
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/FrancophonIA/WiLI-2018 HTTP/11" 200 1480
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/FrancophonIA/WiLI-2018/resolve/d505038b20006eb17994578c1e870d443f1cd5d2/README.md HTTP/11" 200 0
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/FrancophonIA/WiLI-2018/resolve/d505038b20006eb17994578c1e870d443f1cd5d2/.huggingface.yaml HTTP/11" 404 0
2024-10-29 20:34:09 urllib3.connectionpool DEBUG    Starting ne

In [38]:
batch[3].__dict__

{'context': 'أعلنت اليمن قبولها لتدخل أطراف ثالثة لحل المشكلة في حين فضلت السعودية حل المشكلة في الإطار الثنائي وقد اقترحت اليمن تشكيل لجنة عسكرية يمنية ـ سعودية برئاسة عيسى رئيس هيئة الأركان وبإشراف ممثلين من كل من مصر وسوريا للاطلاع ميدانيا على حقيقة الأوضاع والتأكد من عدم وجود الحشود العسكرية السعودية على حدود الأراضي اليمنية لكن الأمير سلطان بن عبد العزيز النائب الثاني لرئيس مجلس الوزراء وزير الدفاع والطيران السعودي صرح بقوله ـ نحن نرحب بكل لقاء أخوى وكل تأييد أخوى لمبادئ السلام والمحبة سواء من إخواننا في سورية أو في مصر لكن العلاقات السعودية ـ اليمنية لا تستوجب الآن دخول أي أخ عربى ',
 'questions': ['# Is the text presented in Pushto?',
  '# Is the primary language of this text Persian?',
  '# Is the content of this text in Tamil?',
  '# Is the text presented in Russian?',
  '# Can we say this is in Dutch?',
  '# Is the primary language of this text Arabic?',
  '# Can we confirm this is in Arabic?',
  '# Is this written in the language Spanish?',
  '# Does this writing reflect the